# Taller de Bank Renege con la Distribucion exponecial

In [13]:
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np
import math

n=1000 #numeros aleatorias generados

def congruencial(x0):
    a = 1664525
    b = 1013904223
    m = 2**32
    xi = x0
    ui =0
    xi = (a*xi + b)%(m)
    ui = float(xi)/float(m)

    return ui,xi
#funcion=congruencial(n)
#fcx=[] #se declara X
#for i in range(n):
 #   for j in range(len(var)-1):
        #if funcion[i]<=dpax[j]:
           #fcx.append(var[j])
            #break

Distribucion exponecial:

In [14]:
def exponencial(landa,x0):
        ui,x0 = congruencial(x0)
        ex=(-math.log(1-ui))/landa
        return ex,x0

Distribucion uniforme:

In [15]:
def uniforme(a,b,x0):
    ui,x0 = congruencial(x0)
    uni = a + ui*(b-a)     
    return uni,x0

Metodo promedio:

In [16]:
def promedio(lista):
    sum=0.0
    for i in range(0,len(lista)):
        sum=sum+lista[i]
        return sum/len(lista)

In [17]:
import random
import simpy

RANDOM_SEED = 42
x0uniform =RANDOM_SEED
x0expon  = RANDOM_SEED


NEW_CUSTOMERS = 5  # Total number of customers - Número total de clientes
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds -Generar nuevos clientes aproximadamente cada x segundos
MIN_PATIENCE = 1  # Min. customer patience - Mini. paciencia de los clientes
MAX_PATIENCE = 3  # Max. customer patience - Max. paciencia de los clientes

numeroDeClientes = 0
numeroClientesAtendidos = 0
numeroClientesAbandonaron = 0
tiempoPromedioEspera = 0
tamanoPromedioFila = []
def source(env, number, interval, counter):
    """Source generates customers randomly"""
    x0 = RANDOM_SEED
    for i in range(number):
        c = customer(env,'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c) 
        ex,x0 =exponencial((1.0/interval),x0)
        t = ex
        yield env.timeout(t)
    print('El numero De clientes es = %d'%(numeroDeClientes))
    print('El numero De clientes atendidos es = %d'%(numeroClientesAtendidos))
    print('El numero De clientes que abandonaron la cola es = %d'%(numeroClientesAbandonaron))
    print('# de personas en la fila: ')
    print(tamanoPromedioFila)
    prom = promedio(tamanoPromedioFila)
    print('# de personas promedio en fila: ')
    prom =  math.ceil(prom)
    print(prom)

def customer(env, name, counter, time_in_bank):
    global x0expon
    global x0uniform
    global numeroDeClientes 
    global numeroClientesAtendidos
    global numeroClientesAbandonaron
    global tamanoPromedioFila
  
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))
    numeroDeClientes +=1
    with counter.request() as req:
        patience,x0uniform = uniforme(MIN_PATIENCE,MIN_PATIENCE,x0uniform)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            if(len(tamanoPromedioFila)==0):
                tamanoPromedioFila.append(1)
            else:
                tamnuevo = tamanoPromedioFila[len(tamanoPromedioFila)- 1]+1
                tamanoPromedioFila.append(tamnuevo)
            ex,x0expon = exponencial((1.0/time_in_bank),x0expon)
            tib = ex            
            yield env.timeout(tib)            
            print('%7.4f %s: Finished' % (env.now, name))
            tamnuevo = (tamanoPromedioFila[len(tamanoPromedioFila) -1]) -1
            tamanoPromedioFila.append(tamnuevo)
            
            numeroClientesAtendidos += 1
        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            numeroClientesAbandonaron += 1

# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 2.9081 Customer01: Here I am
 3.4898 Customer00: Finished
 3.4898 Customer01: Waited  0.582
 3.8307 Customer02: Here I am
 4.5968 Customer01: Finished
 4.5968 Customer02: Waited  0.766
12.4411 Customer03: Here I am
13.4411 Customer03: RENEGED after  1.000
14.9294 Customer02: Finished
14.9586 Customer04: Here I am
14.9586 Customer04: Waited  0.000
17.9795 Customer04: Finished
El numero De clientes es = 5
El numero De clientes atendidos es = 4
El numero De clientes que abandonaron la cola es = 1
# de personas en la fila: 
[1, 0, 1, 0, 1, 0, 1, 0]
# de personas promedio en fila: 
1.0
